# Scripts que devuelve las diferencias entre los ficheros

Importamos

In [133]:
import json
import os
import pandas as pd
from difflib import SequenceMatcher
import jellyfish
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.distance import great_circle
import folium
from folium.plugins import MarkerCluster

In [134]:
#Center coordinates
SF_COORDINATES = (40.4168, -3.7038)

# for speed purposes
MAX_RECORDS = 10

In [135]:
def getDistance_meters(x,y):
    try:
        return great_circle(x, y).kilometers*1000.0
    except:
        return 0

In [136]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/data/')

Creamos un data para los datos de OSM

In [137]:
with open('drinking_water.geojson') as f:
    OSM = json.load(f)
f.close()

In [138]:
OSM['features']

[{'geometry': {'coordinates': [-3.6957228, 40.413707], 'type': 'Point'},
  'id': 'node/167302670',
  'properties': {'@id': 'node/167302670', 'amenity': 'drinking_water'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.6761993, 40.4458451], 'type': 'Point'},
  'id': 'node/297655024',
  'properties': {'@id': 'node/297655024', 'amenity': 'drinking_water'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.6839934, 40.4672427], 'type': 'Point'},
  'id': 'node/297655027',
  'properties': {'@id': 'node/297655027', 'amenity': 'drinking_water'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.6630123, 40.4232182], 'type': 'Point'},
  'id': 'node/297682006',
  'properties': {'@id': 'node/297682006',
   'amenity': 'drinking_water',
   'name': 'Fuente del Berro'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.669598, 40.4812719], 'type': 'Point'},
  'id': 'node/307782191',
  'properties': {'@id': 'node/307782191',
   'amenity': 'drinking_water',
   'name': 'Fuente

In [139]:
ids = [OSM['features'][i]['id'] for i in range(len(OSM['features']))]
lat = [OSM['features'][i]['geometry']['coordinates'][1] for i in range(len(OSM['features']))]
lon = [OSM['features'][i]['geometry']['coordinates'][0] for i in range(len(OSM['features']))]
ayu = ids[:]

In [140]:
dataOSM = pd.DataFrame([ids,lat,lon])

In [141]:
dataOSM = dataOSM.transpose()
dataOSM.columns = ['ids','lat','lon']
dataOSM.lat = dataOSM.lat.astype('float32')
dataOSM.lon = dataOSM.lon.astype('float32')
dataOSM.head(2)

,ids,lat,lon
0,node/167302670,40.413708,-3.695723
1,node/297655024,40.445847,-3.676199


In [142]:
dataOSM['coord'] = list(zip(dataOSM.lat, dataOSM.lon))

In [143]:
dataOSM = dataOSM.sort_values('lat')

In [144]:
AYU = pd.read_csv('InventarioFuentes.csv', encoding='latin-1', delimiter=';')
AYU = AYU[AYU.LATITUD != 'Error']
AYU = AYU[AYU.LONGITUD != 'Error']
AYU.LATITUD = AYU.LATITUD.astype('float32')
AYU.LONGITUD = AYU.LONGITUD.astype('float32')
AYU = AYU.sort_values('LATITUD')
AYU['id_OSM'] = pd.Series()
AYU.head(2)

,CODIGO,ZONA VERDE / VIA PUBLICA,DISTRITO,DIRECCION,COMPLEMENTO DE DIRECCIÓN,COORD. X,COORD. Y,SISTEMA DE COORDENADAS,ESTADO,OBSERVACIONES,FECHA DATOS,LONGITUD,LATITUD,id_OSM
1337,FUE_06-17_0000000113,VP,VILLAVERDE,AV. DE LOS ROSALES 407,NaN,"443914,6707","4464487,999",ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.660224,40.329044,NaN
1285,FUE_06-17_0000000060,ZV,VILLAVERDE,INST. DEP. COL. MARCONI,NaN,"440662,8099","4465018,166",ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.698551,40.333595,NaN


In [145]:
AYU['coord'] = list(zip(AYU.LATITUD, AYU.LONGITUD))

In [146]:
AYU = AYU.reset_index()

In [147]:
dataOSM.coord.iloc[0]

(40.329044, -3.6602237)

In [148]:
getDistance_meters(dataOSM.coord.iloc[1],AYU.coord.iloc[1])

0.08085875116369658

In [149]:
AYU.head(2)

,index,CODIGO,ZONA VERDE / VIA PUBLICA,DISTRITO,DIRECCION,COMPLEMENTO DE DIRECCIÓN,COORD. X,COORD. Y,SISTEMA DE COORDENADAS,ESTADO,OBSERVACIONES,FECHA DATOS,LONGITUD,LATITUD,id_OSM,coord
0,1337,FUE_06-17_0000000113,VP,VILLAVERDE,AV. DE LOS ROSALES 407,NaN,"443914,6707","4464487,999",ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.660224,40.329044,NaN,"(40.329, -3.66022)"
1,1285,FUE_06-17_0000000060,ZV,VILLAVERDE,INST. DEP. COL. MARCONI,NaN,"440662,8099","4465018,166",ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.698551,40.333595,NaN,"(40.3336, -3.69855)"


In [182]:
def get_id(x):
    for i in range(len(ids)):
        point = [lat[i],lon[i]]
        if getDistance_meters(x,point) <5:
            ayu[i] = x
            return ids[i]
    return 'No match'

def get_close(x):
    for i in range(len(ids)):
        point = [lat[i],lon[i]]
        if getDistance_meters(x,point) <30:
            ayu[i] = x
            return ids[i]
    return 'No match'

In [183]:
AYU.id_OSM = AYU.coord.apply(lambda x: get_id(x)) 
AYU_NO = AYU[AYU.id_OSM == 'No match']
AYU_OK = AYU[AYU.id_OSM != 'No match']

In [184]:
AYU_CLOSE = AYU_NO.copy()
AYU_CLOSE.id_OSM = AYU_NO.coord.apply(lambda x: get_close(x))
AYU_NO = AYU_CLOSE[AYU_CLOSE.id_OSM == 'No match']
AYU_CLOSE = AYU_CLOSE[AYU_CLOSE.id_OSM != 'No match']

## Close to edit

In [185]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_CLOSE.id_OSM.values)):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(AYU_CLOSE.id_OSM.values[i], parse_html=True),
        location = (AYU_CLOSE.LATITUD.values[i],AYU_CLOSE.LONGITUD.values[i])).add_to(marker_cluster)
map

## Missing in OSM

In [186]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_NO.id_OSM.values[:100])):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(AYU_NO.id_OSM.values[i], parse_html=True),
        location = (AYU_NO.LATITUD.values[i],AYU_NO.LONGITUD.values[i])).add_to(marker_cluster)
map

## Missing in Ayu website

In [187]:
AYU_MISSING = []
for i in range(len(ids)):
    if isinstance(ayu[i], str):
        AYU_MISSING.append((ids[i],lat[i],lon[i]))

In [188]:
AYU_MISSING = pd.DataFrame(AYU_MISSING)
AYU_MISSING.columns = ['id_OSM','LATITUD','LONGITUD']

In [189]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_MISSING.id_OSM.values)):
    folium.Marker(icon=folium.Icon(color='green'),popup=folium.Popup(AYU_MISSING.id_OSM.values[i], parse_html=True),
        location = (AYU_MISSING.LATITUD.values[i],AYU_MISSING.LONGITUD.values[i])).add_to(marker_cluster)
map

In [211]:
AYU_MISSING = AYU_MISSING.set_index('id_OSM')

In [212]:
mydic = AYU_MISSING.to_dict(orient='index')

In [213]:
mydic.keys()

dict_keys(['node/1504407832', 'node/4340080845', 'node/4344517489', 'node/4333068041', 'node/3507232999', 'node/2624160636', 'node/1440939173', 'node/4332033575', 'node/1460364330', 'node/3361315286', 'node/3973689277', 'node/4395887289', 'node/3497615272', 'node/4439535956', 'node/1436712916', 'node/5102771713', 'node/5126313627', 'node/1802463854', 'node/3500619801', 'node/1743699840', 'node/3697360449', 'node/4663033397', 'node/3497619832', 'node/3514716616', 'node/1436712962', 'node/1440939166', 'node/4340080832', 'node/4347754191', 'node/3697360452', 'node/4332033581', 'node/5086461228', 'node/4667303285', 'node/4065603214', 'node/4333068049', 'node/4333068048', 'node/4403299621', 'node/1992838962', 'node/1228619939', 'node/1828866774', 'node/3912611669', 'node/4376883228', 'node/5014664921', 'node/1992836768', 'node/1464142407', 'node/2623339506', 'node/1802463886', 'node/991793741', 'node/3936936441', 'node/3577606786', 'node/1325487024', 'node/2046863335', 'node/4219016930', 'n

In [216]:
import json
print(json.dumps(mydic, indent=4))

{
    "node/1504407832": {
        "LONGITUD": -3.5909,
        "LATITUD": 40.4874
    },
    "node/4340080845": {
        "LONGITUD": -3.7651854,
        "LATITUD": 40.4543398
    },
    "node/4344517489": {
        "LONGITUD": -3.7219693,
        "LATITUD": 40.4688776
    },
    "node/4333068041": {
        "LONGITUD": -3.7259085,
        "LATITUD": 40.4302336
    },
    "node/3507232999": {
        "LONGITUD": -3.6937871,
        "LATITUD": 40.4976391
    },
    "node/2624160636": {
        "LONGITUD": -3.7556711,
        "LATITUD": 40.4124312
    },
    "node/1440939173": {
        "LONGITUD": -3.7515119,
        "LATITUD": 40.4113367
    },
    "node/4332033575": {
        "LONGITUD": -3.7232446,
        "LATITUD": 40.4577502
    },
    "node/1460364330": {
        "LONGITUD": -3.6561554,
        "LATITUD": 40.4000879
    },
    "node/3361315286": {
        "LONGITUD": -3.7337551,
        "LATITUD": 40.4307789
    },
    "node/3973689277": {
        "LONGITUD": -3.6309041,
       